<a href="https://colab.research.google.com/github/GitLeandroHub/AI_MachineLearning_stock_apple_predict/blob/main/UsingMLtopredictApple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
pd.__version__
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras import regularizers
import plotly.graph_objects as go

#Details of CSV file
filename = "/content/drive/MyDrive/CSV Files/AAPL.csv"
df = pd.read_csv(filename)
print(df.info())

#Drop the data we don't need to use
df['Date'] = pd.to_datetime(df['Date'])
df.set_axis(df['Date'], inplace=True)
df = df.dropna()
df.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'], inplace=True)

cl_data = df['Close'].values
cl_data = cl_data.reshape((len(cl_data), 1))
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(cl_data)

print('Min: %f, Max: %f' % (scaler.data_min_, scaler.data_max_))
close_data = scaler.transform(cl_data)
print(close_data)

#Split train and test data
split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

close_train = close_train.reshape((len(close_train), 1))
close_test = close_test.reshape((len(close_test), 1))

look_back = 5

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=5)
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

#ML Model
model = Sequential()
model.add(LSTM(80, kernel_regularizer=regularizers.l2(0.001), return_sequences=True, activation='relu', input_shape=(look_back,1)))
model.add(LSTM(40, kernel_regularizer=regularizers.l2(0.001), return_sequences=True, activation='relu',))
model.add(LSTM(4, kernel_regularizer=regularizers.l2(0.001), activation='relu',))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 20
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

def predict(num_prediction, model):
  prediction_list = close_data[-look_back:]

  for _ in range(num_prediction):
    x = prediction_list[-look_back:]
    x = x.reshape((1, look_back, 1))
    out = model.predict(x)[0][0]
    prediction_list = np.append(prediction_list, out)
  prediction_list = prediction_list[look_back - 1:]

  return prediction_list

def predict_dates(num_prediction):
  last_date = df['Date'].values[-1]
  prediction_dates = pd.date_range(last_date, periods=num_prediction + 1).tolist()
  return prediction_dates

num_prediction = 10
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

#Create a graph
trace1 = go.Scatter(x = date_train, y = close_train, mode='lines', name = 'Data')
trace2 = go.Scatter(x = date_test, y = prediction, mode = 'lines', name = 'Prediction')
trace3 = go.Scatter(x = date_test, y = close_test, mode = 'lines', name = 'Ground Truth')
trace4 = go.Scatter(x = forecast_dates, y = forecast, mode='lines', name = 'Forecast')
layout = go.Layout(title = "Prediction AAPL 10 days", xaxis = {'title' : "Date"}, yaxis = {'title' : "Close"})

fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
fig.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       251 non-null    object 
 1   Open       251 non-null    float64
 2   High       251 non-null    float64
 3   Low        251 non-null    float64
 4   Close      251 non-null    float64
 5   Adj Close  251 non-null    float64
 6   Volume     251 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.9+ KB
None
Min: 50.119999, Max: 113.902496
[[0.        ]
 [0.03327717]
 [0.00889744]
 [0.00493869]
 [0.02359583]
 [0.03868621]
 [0.03872537]
 [0.04766199]
 [0.0469565 ]
 [0.00846629]
 [0.02355665]
 [0.01442405]
 [0.01979385]
 [0.03343394]
 [0.03237568]
 [0.02046016]
 [0.03413949]
 [0.05017052]
 [0.05009211]
 [0.05365894]
 [0.06357544]
 [0.09058128]
 [0.08862149]
 [0.07161057]
 [0.07611805]
 [0.07925372]
 [0.08736726]
 [0.08027285]
 [0.06761259]
 [0.07149298]
 [0.0674166 ]
 [

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/20
39/39 [==============================] - 4s 8ms/step - loss: 0.1761
Epoch 2/20
39/39 [==============================] - 0s 9ms/step - loss: 0.0687
Epoch 3/20
39/39 [==============================] - 0s 9ms/step - loss: 0.0394
Epoch 4/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0235
Epoch 5/20
39/39 [==============================] - 0s 10ms/step - loss: 0.0153
Epoch 6/20
39/39 [==============================] - 0s 9ms/step - loss: 0.0109
Epoch 7/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0093
Epoch 8/20
39/39 [==============================] - 0s 9ms/step - loss: 0.0081
Epoch 9/20
39/39 [==============================] - 0s 9ms/step - loss: 0.0070
Epoch 10/20
39/39 [==============================] - 0s 9ms/step - loss: 0.0070
Epoch 11/20
39/39 [==============================] - 0s 9ms/step - loss: 0.0063
Epoch 12/20
39/39 [==============================] - 0s 9ms/step - loss: 0.0052
Epoch 13/20
39/39 [=============================

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.

